### cria tabelas para API

In [6]:
import mysql.connector
from mysql.connector import errorcode

class mysqlserver:
    # variaveis de entrada
    def __init__(self,user, password, host, database):
        self.__user = user
        self.__password = password
        self.__host = host
        self.__database = database

    def create_server_connection(self):
        conn = None
        try:
            conn = mysql.connector.connect(user=self.__user,password=self.__password,host=self.__host,database=self.__database)
            print("Conectado no banco MYSQL")
        except mysql.connector.Error as err:
            print(f"Error: '{err}'")
        return conn

    def create_table(self, conn):
        cur = conn.cursor()
        try:
            cur.execute(""" CREATE TABLE IF NOT EXISTS candidate(
                            candidate_id VARCHAR(255) PRIMARY KEY,
                            candidate_name VARCHAR(255),
                            party_affiliation VARCHAR(255),
                            biography TEXT,
                            campaign_platform TEXT,
                            photo_url TEXT) 
                        """)
            cur.execute(""" CREATE TABLE IF NOT EXISTS voters (
                            voter_id VARCHAR(255) primary key,
                            voter_name VARCHAR(255),
                            date_of_birth VARCHAR(255),
                            gender VARCHAR(255),
                            nationality VARCHAR(255),
                            registration_number VARCHAR(255),
                            address_street VARCHAR(255),
                            address_city VARCHAR(255),
                            address_state VARCHAR(255),
                            address_country VARCHAR(255),
                            address_postcode VARCHAR(255),
                            email VARCHAR(255),
                            phone_number VARCHAR(255),
                            cell_number VARCHAR(255),
                            picture TEXT,
                            registered_age int
            )""")
            cur.execute("""CREATE TABLE IF NOT EXISTS votes (
                        voter_id VARCHAR(255) UNIQUE,
                        candidate_id VARCHAR(255),
                        voting_time TIMESTAMP,
                        vote int DEFAULT 1,
                        primary key (voter_id, candidate_id)
                        ) 
                        """)
            conn.commit()
        except mysql.connector.Error as err:
                print(f"Error: '{err}'") 

 

In [7]:
x = mysqlserver('root','mysql','localhost','mydesenv')
conn = x.create_server_connection()
x.create_table(conn)

Conectado no banco MYSQL


### insert de dados da API

In [8]:
import mysql.connector
from mysql.connector import errorcode
import random
import requests
from confluent_kafka import SerializingProducer
import simplejson as json

BASE_URL = 'https://randomuser.me/api/?nat=gb'
PARTIES = ["Management Party", "Savior Party", "Tech Republic Party"]
random.seed(42)


def generate_voter_data():
    response = requests.get(BASE_URL)
    if response.status_code == 200:
        user_data = response.json()['results'][0]
        return {
            "voter_id": user_data['login']['uuid'],
            "voter_name": f"{user_data['name']['first']} {user_data['name']['last']}",
            "date_of_birth": user_data['dob']['date'],
            "gender": user_data['gender'],
            "nationality": user_data['nat'],
            "registration_number": user_data['login']['username'],
            "address": {
                "street": f"{user_data['location']['street']['number']} {user_data['location']['street']['name']}",
                "city": user_data['location']['city'],
                "state": user_data['location']['state'],
                "country": user_data['location']['country'],
                "postcode": user_data['location']['postcode']
            },
            "email": user_data['email'],
            "phone_number": user_data['phone'],
            "cell_number": user_data['cell'],
            "picture": user_data['picture']['large'],
            "registered_age": user_data['registered']['age']
        }
    else:
        return "Error fetching data"

def generate_candidate_data(candidate_number, total_parties):
    response = requests.get(BASE_URL + '&gender=' + ('female' if candidate_number % 2 == 1 else 'male'))
    if response.status_code == 200:
        user_data = response.json()['results'][0]
        return {
            "candidate_id": user_data['login']['uuid'],
            "candidate_name": f"{user_data['name']['first']} {user_data['name']['last']}",
            "party_affiliation": PARTIES[candidate_number % total_parties],
            "biography": "A brief bio of the candidate.",
            "campaign_platform": "Key campaign promises or platform.",
            "photo_url": user_data['picture']['large']
        }
    else:
        return "Error fetching data"

def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')
    

class mysqlserver:
    # variaveis de entrada
    def __init__(self,user, password, host, database):
        self.__user = user
        self.__password = password
        self.__host = host
        self.__database = database



           
    def create_server_connection(self):
        conn = None
        try:
            conn = mysql.connector.connect(user=self.__user,password=self.__password,host=self.__host,database=self.__database)
            print("Conectado no banco MYSQL")
        except mysql.connector.Error as err:
            print(f"Error: '{err}'")
        return conn
    
    def insert_table(self,conn):
        cur = conn.cursor()
        for i in range(20):
            candidate = generate_candidate_data(i, total_parties=3)
            print(candidate)
            cur.execute("""
                        INSERT INTO candidate (candidate_id, candidate_name, party_affiliation, biography, campaign_platform, photo_url)
                        VALUES (%s, %s, %s, %s, %s, %s)
                        """, (
                candidate['candidate_id'], candidate['candidate_name'], candidate['party_affiliation'], candidate['biography'],
                candidate['campaign_platform'], candidate['photo_url']))
        for i in range(30):
            voter = generate_voter_data()
            print(voter)
            cur.execute("""
                                INSERT INTO voters (voter_id, voter_name, date_of_birth, gender, nationality, registration_number, address_street, address_city, address_state, address_country, address_postcode, email, phone_number, cell_number, picture, registered_age)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)
                                """,
                        (voter["voter_id"], voter['voter_name'], voter['date_of_birth'], voter['gender'],
                        voter['nationality'], voter['registration_number'], voter['address']['street'],
                        voter['address']['city'], voter['address']['state'], voter['address']['country'],
                        voter['address']['postcode'], voter['email'], voter['phone_number'],
                        voter['cell_number'], voter['picture'], voter['registered_age'])
                        )
            conn.commit()
           # Kafka Topics
            producer = SerializingProducer({'bootstrap.servers': 'localhost:9092'})

            #delivery_report = delivery_report()
 
            voters_topic = 'voters_topic'
            candidates_topic = 'candidates_topic'
            producer.produce(
            voters_topic,
            key=voter["voter_id"],
            value=json.dumps(voter)
        )
        producer.flush()



In [9]:
x = mysqlserver('root','mysql','localhost','mydesenv')
conn = x.create_server_connection()
x.insert_table(conn)

Conectado no banco MYSQL
{'candidate_id': '95812e15-c7c5-4f3e-bd95-f9b0b2df7b54', 'candidate_name': 'Clifton Mccoy', 'party_affiliation': 'Management Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/men/71.jpg'}
{'candidate_id': 'e2b5792c-d06e-406a-b85e-35878c04190a', 'candidate_name': 'Katherine Ward', 'party_affiliation': 'Savior Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/women/45.jpg'}
{'candidate_id': '0e340a93-7f92-44fa-bd94-9364b3843e2d', 'candidate_name': 'Danny Austin', 'party_affiliation': 'Tech Republic Party', 'biography': 'A brief bio of the candidate.', 'campaign_platform': 'Key campaign promises or platform.', 'photo_url': 'https://randomuser.me/api/portraits/men/40.jpg'}
{'candidate_id': '5f379962-6d59-4d40-b8ac-3bc71c4055b5', 'candidate_nam

%4|1704250446.549|TERMINATE|rdkafka#producer-3| [thrd:app]: Producer terminating with 1 message (538 bytes) still in queue or transit: use flush() to wait for outstanding message delivery
%4|1704250446.741|TERMINATE|rdkafka#producer-4| [thrd:app]: Producer terminating with 1 message (527 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


{'voter_id': 'bcea05c4-0155-4484-974f-87f50cbc1a32', 'voter_name': 'Maya Rodriquez', 'date_of_birth': '1997-12-14T15:37:30.692Z', 'gender': 'female', 'nationality': 'GB', 'registration_number': 'yellowleopard512', 'address': {'street': '10 London Road', 'city': 'Stevenage', 'state': 'Dorset', 'country': 'United Kingdom', 'postcode': 'AG7I 3TW'}, 'email': 'maya.rodriquez@example.com', 'phone_number': '015396 01587', 'cell_number': '07194 148007', 'picture': 'https://randomuser.me/api/portraits/women/2.jpg', 'registered_age': 14}
{'voter_id': '9f239071-f9a1-40ea-9e6c-5bbe7dd3c560', 'voter_name': 'Lillian Willis', 'date_of_birth': '1984-11-25T14:37:18.691Z', 'gender': 'female', 'nationality': 'GB', 'registration_number': 'smallgoose735', 'address': {'street': '5848 Alexander Road', 'city': 'Lincoln', 'state': 'Buckinghamshire', 'country': 'United Kingdom', 'postcode': 'X9 5LF'}, 'email': 'lillian.willis@example.com', 'phone_number': '0151 856 9829', 'cell_number': '07258 696332', 'picture

%4|1704250446.931|TERMINATE|rdkafka#producer-5| [thrd:app]: Producer terminating with 1 message (533 bytes) still in queue or transit: use flush() to wait for outstanding message delivery
%4|1704250447.109|TERMINATE|rdkafka#producer-6| [thrd:app]: Producer terminating with 1 message (541 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


{'voter_id': '993acb40-9924-412f-ac54-0013b7350db2', 'voter_name': 'Laura Mcdonalid', 'date_of_birth': '1951-05-20T20:32:20.132Z', 'gender': 'female', 'nationality': 'GB', 'registration_number': 'crazybird142', 'address': {'street': '7594 School Lane', 'city': 'Durham', 'state': 'Avon', 'country': 'United Kingdom', 'postcode': 'MY7X 8XA'}, 'email': 'laura.mcdonalid@example.com', 'phone_number': '016974 77750', 'cell_number': '07425 989886', 'picture': 'https://randomuser.me/api/portraits/women/68.jpg', 'registered_age': 17}
{'voter_id': 'f95c2af6-03e0-4549-af52-f3bb26b5e5f9', 'voter_name': 'Zoe Martinez', 'date_of_birth': '1985-06-23T13:42:25.860Z', 'gender': 'female', 'nationality': 'GB', 'registration_number': 'bigwolf285', 'address': {'street': '1669 Queen Street', 'city': 'Winchester', 'state': 'Gloucestershire', 'country': 'United Kingdom', 'postcode': 'Y9E 0BJ'}, 'email': 'zoe.martinez@example.com', 'phone_number': '016973 99748', 'cell_number': '07997 151761', 'picture': 'https:

...

%4|1704250451.153|TERMINATE|rdkafka#producer-26| [thrd:app]: Producer terminating with 1 message (558 bytes) still in queue or transit: use flush() to wait for outstanding message delivery
%4|1704250451.342|TERMINATE|rdkafka#producer-27| [thrd:app]: Producer terminating with 1 message (526 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


{'voter_id': '28783ce2-561d-48c8-8c83-d89ef3908d22', 'voter_name': 'Calvin Lucas', 'date_of_birth': '1973-04-26T00:01:52.120Z', 'gender': 'male', 'nationality': 'GB', 'registration_number': 'orangeduck843', 'address': {'street': '5598 Green Lane', 'city': 'St Davids', 'state': 'West Midlands', 'country': 'United Kingdom', 'postcode': 'RP7 1SQ'}, 'email': 'calvin.lucas@example.com', 'phone_number': '017684 58288', 'cell_number': '07675 055908', 'picture': 'https://randomuser.me/api/portraits/men/88.jpg', 'registered_age': 9}
{'voter_id': '2bc186a6-e180-4072-acba-f5309858fe81', 'voter_name': 'Barbara Perkins', 'date_of_birth': '1956-03-26T19:15:25.721Z', 'gender': 'female', 'nationality': 'GB', 'registration_number': 'purplewolf402', 'address': {'street': '5044 Main Street', 'city': 'Portsmouth', 'state': 'Grampian', 'country': 'United Kingdom', 'postcode': 'TH03 7FZ'}, 'email': 'barbara.perkins@example.com', 'phone_number': '0131 769 1253', 'cell_number': '07269 714384', 'picture': 'htt

%4|1704250451.600|TERMINATE|rdkafka#producer-28| [thrd:app]: Producer terminating with 1 message (525 bytes) still in queue or transit: use flush() to wait for outstanding message delivery
%4|1704250451.799|TERMINATE|rdkafka#producer-29| [thrd:app]: Producer terminating with 1 message (529 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


{'voter_id': '82c4104e-4196-48ab-99f1-193694a4f582', 'voter_name': 'Kelly Carter', 'date_of_birth': '1958-03-10T09:51:46.164Z', 'gender': 'female', 'nationality': 'GB', 'registration_number': 'bigbutterfly631', 'address': {'street': '1099 Broadway', 'city': 'Dundee', 'state': 'Warwickshire', 'country': 'United Kingdom', 'postcode': 'L5E 3SL'}, 'email': 'kelly.carter@example.com', 'phone_number': '016973 90738', 'cell_number': '07467 012672', 'picture': 'https://randomuser.me/api/portraits/women/58.jpg', 'registered_age': 13}
{'voter_id': '87f1901e-b906-4ca3-bcbb-a2b6e5c0f638', 'voter_name': 'Jimmy Jordan', 'date_of_birth': '1954-08-24T15:29:13.073Z', 'gender': 'male', 'nationality': 'GB', 'registration_number': 'sadpanda713', 'address': {'street': '634 Park Lane', 'city': 'Bangor', 'state': 'Cambridgeshire', 'country': 'United Kingdom', 'postcode': 'T1 8RP'}, 'email': 'jimmy.jordan@example.com', 'phone_number': '015394 81490', 'cell_number': '07663 191453', 'picture': 'https://randomus

%4|1704250451.995|TERMINATE|rdkafka#producer-30| [thrd:app]: Producer terminating with 1 message (539 bytes) still in queue or transit: use flush() to wait for outstanding message delivery
%4|1704250452.167|TERMINATE|rdkafka#producer-31| [thrd:app]: Producer terminating with 1 message (530 bytes) still in queue or transit: use flush() to wait for outstanding message delivery


### insert voter

In [10]:
import random
import time
from datetime import datetime
import mysql.connector
from mysql.connector import errorcode
import simplejson as json
from confluent_kafka import Consumer, KafkaException, KafkaError, SerializingProducer


conf = {
    'bootstrap.servers': 'localhost:9092',
}

consumer = Consumer(conf | {
    'group.id': 'voting-group',
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': False
})

producer = SerializingProducer(conf)

def consume_messages():
    result = []
    consumer.subscribe(['candidates_topic'])
    try:
        while True:
            msg = consumer.poll(timeout=1.0)
            if msg is None:
                continue
            elif msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                else:
                    print(msg.error())
                    break
            else:
                result.append(json.loads(msg.value().decode('utf-8')))
                if len(result) == 3:
                    return result

            # time.sleep(5)
    except KafkaException as e:
        print(e)

In [11]:
df = consume_messages()
print(df)

KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Subscribed topic not available: candidates_topic: Broker: Unknown topic or partition"}
None
